In [ ]:
##
#Import Packages
import numpy as np
import nibabel as nib
import sys
import pickle

##
#SBI Specific Packages
import torch
from sbi import analysis as analysis
from sbi import utils as utils

In [ ]:
##
#Define Path to Code database
DirPath = '/PATH/TO/bin/'

#Define Path for Posterior Object
InputPath = '/PATH/TO/Posterior.pkl'

#Define Path to Example Data
DataPath = '/PATH/TO/ExampleData/'

#Define Output Path
OutputPath = '/PATH/TO/Output/'

In [ ]:
##
#Import Custom Functions
sys.path.append(DirPath)
from ImportData import *
from FreedAnalytical import *

In [ ]:
##
#Load Posterior
with open(InputPath, "rb") as handle:
    posterior = pickle.load(handle)

In [ ]:
##
#Load Data
    # Data - DW-SSFP Data (4D)
    # T1Map - T1 Map (ms, 3D)
    # T2Map - T2 Map (ms, 3D)
    # B1Map - B1 Map (Normalised, 3D)
    # Mask - Mask (Binary, 3D)
    # noisefloor - noisefloor estimate (1xn)
    # bvecs - bvectors (3xn)
    # FlipAngles - Flip Angles (degrees) (1xn)
    # tau - Diffusion Gradient Duration (seconds) (1xn), 
    # G - Diffusion Gradient Duration (G/cm - Equivalent to mT/m Divided by 10) (1xn)
    # TRs - Repetition Times (seconds) (1xn)
    # b0s - Array Defining b0 Locations (b0 = 1, dwi = 0) (1xn)
Data, T1Map, T2Map, B1Map, Mask, noisefloor, bvecs, FlipAngles, tau, G, TRs, b0s = ImportDataDWSSFP(DataPath)

In [ ]:
##
#Estimate Mean Noisefloor
Noisefloor_Mean = np.mean(noisefloor)

In [ ]:
##
#Normalise Data by S0 (Estimate S0 per Flip Angle) & Perform Noisefloor Correction 

##
#Define unique locations
Values, Index = np.unique(FlipAngles, return_index=True)

##
#Normalised Data for Each Flip angle
for idx, k in enumerate(Index):
    #Calculate Theoretical Signal Amplitude
    b0 = FreedDWSSFP(G[k], tau[k], TRs[k], FlipAngles[k]*B1Map.data, 0, T1Map.data, T2Map.data)
    #Identify S0 (Incorporating Noisefloor Contribution)
    S0 = np.abs(np.mean(Data.data[:,:,:,(b0s == 1) & (FlipAngles == Values[idx])],axis = 3)**2-Noisefloor_Mean**2)**0.5/b0*Mask.data
    #Divide by S0 & Remove NoiseFloor
    Data.data[:,:,:,FlipAngles == Values[idx]] = np.abs(Data.data[:,:,:,FlipAngles == Values[idx]]**2-Noisefloor_Mean**2)**0.5/S0[:,:,:,np.newaxis]

In [ ]:
##
#Perform Posterior Estimation (NPE)

##
#Define Number of Samples
Samples = 100

##
#Initialise Output Array (Tensor Estimates & Voxels where Posterior Couldn't be Evaluated)
Tensor = np.zeros((*T1Map.data.shape,Samples,6))
TensorErr = np.zeros(T1Map.data.shape)

##
#Perform Posterior Estimation for all Voxels in Dataset
for k in range(Data.data.shape[0]):
    for l in range(Data.data.shape[1]):
        for m in range(Data.data.shape[2]):
            if Mask.data[k,l,m] == 0:
                continue
            #Concatenate Data 
            DataVec = torch.from_numpy(np.concatenate((Data.data[k,l,m,:],[B1Map.data[k,l,m]/100],[T1Map.data[k,l,m]/100000],[T2Map.data[k,l,m]/10000])))[np.newaxis,:]
            #Perform Posterior Estimation
            Tensor[k,l,m,:,:] = posterior.sample((Samples,), x=DataVec, show_progress_bars = False)
    print(k)

In [ ]:
##
#Save Mean of Tensor Estimates
nib.save(nib.Nifti1Image(np.mean(Tensor[:,:,:,:,0],axis=3),Data.aff),''.join([OutputPath, 'D11_Mean.nii.gz']))
nib.save(nib.Nifti1Image(np.mean(Tensor[:,:,:,:,1],axis=3),Data.aff),''.join([OutputPath, 'D22_Mean.nii.gz']))
nib.save(nib.Nifti1Image(np.mean(Tensor[:,:,:,:,2],axis=3),Data.aff),''.join([OutputPath, 'D33_Mean.nii.gz']))
nib.save(nib.Nifti1Image(np.mean(Tensor[:,:,:,:,3],axis=3),Data.aff),''.join([OutputPath, 'D12_Mean.nii.gz']))
nib.save(nib.Nifti1Image(np.mean(Tensor[:,:,:,:,4],axis=3),Data.aff),''.join([OutputPath, 'D13_Mean.nii.gz']))
nib.save(nib.Nifti1Image(np.mean(Tensor[:,:,:,:,5],axis=3),Data.aff),''.join([OutputPath, 'D23_Mean.nii.gz']))

In [ ]:
##
#Save Full Posterior Outputs of Tensor Components
nib.save(nib.Nifti1Image(Tensor[:,:,:,:,0],Data.aff),''.join([OutputPath, 'D11_Samples.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,:,1],Data.aff),''.join([OutputPath, 'D22_Samples.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,:,2],Data.aff),''.join([OutputPath, 'D33_Samples.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,:,3],Data.aff),''.join([OutputPath, 'D12_Samples.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,:,4],Data.aff),''.join([OutputPath, 'D13_Samples.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,:,5],Data.aff),''.join([OutputPath, 'D23_Samples.nii.gz']))

In [ ]:
##
#Convert the Diffusivity Outputs to Eigenvalue/Eigenvector/FA Outputs in FSL via
#fslmerge -t dti D11_Mean.nii.gz D12_Mean.nii.gz D13_Mean.nii.gz D22_Mean.nii.gz D23_Mean.nii.gz D33_Mean.nii.gz
#fslmaths dti.nii.gz -tensor_decomp dti